In [ ]:
# default_exp core

# MLDaTo

> API details. MLDaTo stand for machine learning on any data in an automated fasion

In [1]:
#hide
from nbdev.showdoc import *

In [3]:
def our_first_function():
    print('hello')